In [35]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
import tensorflow as tf

In [97]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [19]:
iris = load_iris()
X = iris.data[:,2:4]
X = iris.data[:, (2, 3)]
y = (iris.target == 0)

In [24]:
per_clf = Perceptron(random_state=42)
per_clf.fit(X,y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=42, shuffle=True,
      verbose=0, warm_start=False)

In [33]:
y_pred = per_clf.predict([[2,.5]])

In [34]:
y_pred

array([ True], dtype=bool)

## Now in tensorflow

In [135]:
import tensorflow as tf

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [136]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [140]:
def neuron_layer(X, n_neurons,name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs,n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X,W) + b
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z
        

In [141]:
from tensorflow.contrib.layers import fully_connected

In [142]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X,n_hidden1,"hidden1",activation="relu")
    hidden2 = neuron_layer(hidden1,n_hidden2,"hidden2",activation="relu")
    logits = neuron_layer(hidden2,n_outputs,"outputs")
    

In [143]:
from tensorflow.contrib.layers import fully_connected

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [144]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [145]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [146]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Training

In [147]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data/")

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [148]:
n_epochs = 40
batch_size = 50
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [149]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch,y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./my_model_final.ckpt")
    

0 Train accuracy: 0.94 Test accuracy: 0.8997
1 Train accuracy: 0.92 Test accuracy: 0.9157
2 Train accuracy: 0.92 Test accuracy: 0.9188
3 Train accuracy: 0.92 Test accuracy: 0.9186
4 Train accuracy: 0.92 Test accuracy: 0.9212
5 Train accuracy: 0.94 Test accuracy: 0.9217
6 Train accuracy: 0.86 Test accuracy: 0.92
7 Train accuracy: 0.88 Test accuracy: 0.9214
8 Train accuracy: 0.92 Test accuracy: 0.9234
9 Train accuracy: 0.92 Test accuracy: 0.9211
10 Train accuracy: 0.88 Test accuracy: 0.9225
11 Train accuracy: 0.98 Test accuracy: 0.9225
12 Train accuracy: 0.9 Test accuracy: 0.9244
13 Train accuracy: 0.98 Test accuracy: 0.925
14 Train accuracy: 0.84 Test accuracy: 0.9203
15 Train accuracy: 0.94 Test accuracy: 0.9252
16 Train accuracy: 0.98 Test accuracy: 0.923
17 Train accuracy: 0.98 Test accuracy: 0.9234
18 Train accuracy: 0.92 Test accuracy: 0.9215
19 Train accuracy: 0.96 Test accuracy: 0.9237
20 Train accuracy: 0.98 Test accuracy: 0.926
21 Train accuracy: 0.9 Test accuracy: 0.9246
22 Tr

In [150]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = mnist.test.images[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

In [151]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", mnist.test.labels[:20])

Predicted classes: [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
